In [1]:
!pip install pyspark

Looking in indexes: https://artifactory.dataxu.net/artifactory/api/pypi/pypi-virtual/simple/


In [5]:
from pyspark.context import SparkContext
sc = SparkContext('local', 'test')

In [6]:
sc

<SparkContext master=local appName=test>

In [7]:
from pyspark.sql import SQLContext

spark = SQLContext(sc)

In [9]:
spark

In [13]:
df = spark.read.csv("StudentsPerformance.csv", header=True)

In [15]:
df.show(3)

+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|   lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|standard|                   none|        72|           72|           74|
|female|       group C|               some college|standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|standard|                   none|        90|           95|           93|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
only showing top 3 rows



In [18]:
reduced_df = df.select(["gender","math score"])

In [20]:
reduced_df.show(2)

+------+----------+
|gender|math score|
+------+----------+
|female|        72|
|female|        69|
+------+----------+
only showing top 2 rows



In [21]:
from pyspark.sql import functions as F


In [26]:
df.select([F.col("gender"),F.col("math score").alias("math"),
           (F.col("math score") + F.col("reading score")).alias("sum_math_writing")]).show(2)

+------+----+----------------+
|gender|math|sum_math_writing|
+------+----+----------------+
|female|  72|           144.0|
|female|  69|           159.0|
+------+----+----------------+
only showing top 2 rows



In [ ]:
+------+---------+----+
|gender|   course|math|
+------+---------+----+

In [35]:
clean_df = df.select([F.col("gender"),F.col("test preparation course").alias("course"),F.col("math score").alias("math")])
clean_df.show(2)

+------+---------+----+
|gender|   course|math|
+------+---------+----+
|female|     none|  72|
|female|completed|  69|
+------+---------+----+
only showing top 2 rows



In [39]:
clean_df.where(F.col("math")>90).show()

+------+---------+----+
|gender|   course|math|
+------+---------+----+
|  male|     none|  97|
|  male|completed|  98|
|female|completed|  99|
|  male|completed|  91|
|  male|completed| 100|
|female|completed|  96|
|  male|     none|  94|
|female|completed|  97|
|  male|     none|  92|
|female|     none|  99|
|  male|completed|  97|
|  male|completed|  99|
|female|     none| 100|
|female|     none| 100|
|  male|     none|  91|
|female|completed|  94|
|female|completed|  95|
|female|     none|  91|
|  male|completed|  97|
|female|completed|  92|
+------+---------+----+
only showing top 20 rows



In [49]:
clean_df.where((F.col("math")>90) & (F.col('course') == 'completed')).show()

+------+---------+----+
|gender|   course|math|
+------+---------+----+
|  male|completed|  98|
|female|completed|  99|
|  male|completed|  91|
|  male|completed| 100|
|female|completed|  96|
|female|completed|  97|
|  male|completed|  97|
|  male|completed|  99|
|female|completed|  94|
|female|completed|  95|
|  male|completed|  97|
|female|completed|  92|
|  male|completed|  96|
|female|completed|  92|
|female|completed|  92|
|  male|completed|  94|
|  male|completed| 100|
|  male|completed| 100|
|female|completed|  94|
|  male|completed|  93|
+------+---------+----+
only showing top 20 rows



In [44]:
clean_df.groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|  518|
|  male|  482|
+------+-----+



In [45]:
clean_df.groupby("gender").mean().show()

+------+
|gender|
+------+
|female|
|  male|
+------+



In [46]:
clean_df.schema

StructType(List(StructField(gender,StringType,true),StructField(course,StringType,true),StructField(math,StringType,true)))

In [50]:
from pyspark.sql.types import *

custom_schema = StructType([StructField("gender",StringType(),True),
                     StructField("origin",StringType(),True),
                     StructField("education",StringType(),True),
                     StructField("lunch",StringType(),True),
                     StructField("course",StringType(),True),
                     StructField("math",IntegerType(),True),
                     StructField("reading",IntegerType(),True),
                     StructField("writing",IntegerType(),True)])

In [52]:
df = spark.read.csv("StudentsPerformance.csv", header=True, schema=custom_schema)

In [53]:
df.show(2)

+------+-------+-----------------+--------+---------+----+-------+-------+
|gender| origin|        education|   lunch|   course|math|reading|writing|
+------+-------+-----------------+--------+---------+----+-------+-------+
|female|group B|bachelor's degree|standard|     none|  72|     72|     74|
|female|group C|     some college|standard|completed|  69|     90|     88|
+------+-------+-----------------+--------+---------+----+-------+-------+
only showing top 2 rows



In [54]:
df.schema

StructType(List(StructField(gender,StringType,true),StructField(origin,StringType,true),StructField(education,StringType,true),StructField(lunch,StringType,true),StructField(course,StringType,true),StructField(math,IntegerType,true),StructField(reading,IntegerType,true),StructField(writing,IntegerType,true)))

In [55]:
df.groupby("gender").mean().show()

+------+------------------+-----------------+-----------------+
|gender|         avg(math)|     avg(reading)|     avg(writing)|
+------+------------------+-----------------+-----------------+
|female|63.633204633204635|72.60810810810811|72.46718146718146|
|  male| 68.72821576763485|65.47302904564316|63.31120331950208|
+------+------------------+-----------------+-----------------+



In [62]:
mean_df = df.select(["gender","math"]).groupby("gender").mean()
mean_df.show()

+------+------------------+
|gender|         avg(math)|
+------+------------------+
|female|63.633204633204635|
|  male| 68.72821576763485|
+------+------------------+



In [57]:
df.groupby("gender").sum().show()

+------+---------+------------+------------+
|gender|sum(math)|sum(reading)|sum(writing)|
+------+---------+------------+------------+
|female|    32962|       37611|       37538|
|  male|    33127|       31558|       30516|
+------+---------+------------+------------+



In [60]:
df.sort("math").show(3)

+------+-------+----------------+------------+------+----+-------+-------+
|gender| origin|       education|       lunch|course|math|reading|writing|
+------+-------+----------------+------------+------+----+-------+-------+
|female|group C|some high school|free/reduced|  none|   0|     17|     10|
|female|group B|     high school|free/reduced|  none|   8|     24|     23|
|female|group B|some high school|free/reduced|  none|  18|     32|     28|
+------+-------+----------------+------------+------+----+-------+-------+
only showing top 3 rows



In [64]:
joined_df = df.join(mean_df, on='gender')
joined_df.show(5)

+------+-------+------------------+------------+---------+----+-------+-------+------------------+
|gender| origin|         education|       lunch|   course|math|reading|writing|         avg(math)|
+------+-------+------------------+------------+---------+----+-------+-------+------------------+
|female|group B| bachelor's degree|    standard|     none|  72|     72|     74|63.633204633204635|
|female|group C|      some college|    standard|completed|  69|     90|     88|63.633204633204635|
|female|group B|   master's degree|    standard|     none|  90|     95|     93|63.633204633204635|
|  male|group A|associate's degree|free/reduced|     none|  47|     57|     44| 68.72821576763485|
|  male|group C|      some college|    standard|     none|  76|     78|     75| 68.72821576763485|
+------+-------+------------------+------------+---------+----+-------+-------+------------------+
only showing top 5 rows



In [70]:
stats_df = joined_df.select([F.col("gender"),F.col("education"),(F.col("math")-F.col("avg(math)")).alias("mean_diff")])
stats_df.show(4)

+------+------------------+------------------+
|gender|         education|         mean_diff|
+------+------------------+------------------+
|female| bachelor's degree| 8.366795366795365|
|female|      some college| 5.366795366795365|
|female|   master's degree|26.366795366795365|
|  male|associate's degree|-21.72821576763485|
+------+------------------+------------------+
only showing top 4 rows



In [72]:
stats_df.groupby("education").mean().show()

+------------------+------------------+
|         education|    avg(mean_diff)|
+------------------+------------------+
|  some high school|-2.640807872477595|
|associate's degree| 1.816925185490848|
|       high school|-4.146934917448935|
| bachelor's degree| 3.381832549899924|
|   master's degree| 4.126367297441215|
|      some college|1.0603298689260598|
+------------------+------------------+



In [73]:
df.createOrReplaceTempView("students")

In [75]:
result_df = spark.sql("select education, math from students")
result_df.show()

+------------------+----+
|         education|math|
+------------------+----+
| bachelor's degree|  72|
|      some college|  69|
|   master's degree|  90|
|associate's degree|  47|
|      some college|  76|
|associate's degree|  71|
|      some college|  88|
|      some college|  40|
|       high school|  64|
|       high school|  38|
|associate's degree|  58|
|associate's degree|  40|
|       high school|  65|
|      some college|  78|
|   master's degree|  50|
|  some high school|  69|
|       high school|  88|
|  some high school|  18|
|   master's degree|  46|
|associate's degree|  54|
+------------------+----+
only showing top 20 rows



In [79]:
spark.sql("select education, math from students where math>90 and course='completed'").show()

+------------------+----+
|         education|math|
+------------------+----+
|      some college|  98|
| bachelor's degree|  99|
|associate's degree|  91|
|associate's degree| 100|
| bachelor's degree|  96|
|  some high school|  97|
|associate's degree|  97|
|      some college|  99|
|associate's degree|  94|
|associate's degree|  95|
|associate's degree|  97|
|  some high school|  92|
| bachelor's degree|  96|
| bachelor's degree|  92|
| bachelor's degree|  92|
| bachelor's degree|  94|
|      some college| 100|
|      some college| 100|
|   master's degree|  94|
|      some college|  93|
+------------------+----+
only showing top 20 rows



In [87]:
spark.sql("select education, avg(math) from students where math>90 and course='completed' group by education").show()

+------------------+-----------------+
|         education|        avg(math)|
+------------------+-----------------+
|  some high school|94.33333333333333|
|associate's degree| 95.0909090909091|
| bachelor's degree|94.77777777777777|
|   master's degree|             92.5|
|      some college|96.83333333333333|
+------------------+-----------------+



In [83]:
stats_df.cache()

DataFrame[gender: string, education: string, mean_diff: double]

In [84]:
stats_df.show()

+------+------------------+-------------------+
|gender|         education|          mean_diff|
+------+------------------+-------------------+
|female| bachelor's degree|  8.366795366795365|
|female|      some college|  5.366795366795365|
|female|   master's degree| 26.366795366795365|
|  male|associate's degree| -21.72821576763485|
|  male|      some college|   7.27178423236515|
|female|associate's degree|  7.366795366795365|
|female|      some college| 24.366795366795365|
|  male|      some college| -28.72821576763485|
|  male|       high school|  -4.72821576763485|
|female|       high school|-25.633204633204635|
|  male|associate's degree| -10.72821576763485|
|  male|associate's degree| -28.72821576763485|
|female|       high school| 1.3667953667953654|
|  male|      some college|   9.27178423236515|
|female|   master's degree|-13.633204633204635|
|female|  some high school|  5.366795366795365|
|  male|       high school|  19.27178423236515|
|female|  some high school|-45.633204633

In [86]:
stats_df.show()

+------+------------------+-------------------+
|gender|         education|          mean_diff|
+------+------------------+-------------------+
|female| bachelor's degree|  8.366795366795365|
|female|      some college|  5.366795366795365|
|female|   master's degree| 26.366795366795365|
|  male|associate's degree| -21.72821576763485|
|  male|      some college|   7.27178423236515|
|female|associate's degree|  7.366795366795365|
|female|      some college| 24.366795366795365|
|  male|      some college| -28.72821576763485|
|  male|       high school|  -4.72821576763485|
|female|       high school|-25.633204633204635|
|  male|associate's degree| -10.72821576763485|
|  male|associate's degree| -28.72821576763485|
|female|       high school| 1.3667953667953654|
|  male|      some college|   9.27178423236515|
|female|   master's degree|-13.633204633204635|
|female|  some high school|  5.366795366795365|
|  male|       high school|  19.27178423236515|
|female|  some high school|-45.633204633

In [88]:
stats_df.createOrReplaceTempView("stats")


In [90]:
spark.sql("select * from stats").show()

+------+------------------+-------------------+
|gender|         education|          mean_diff|
+------+------------------+-------------------+
|female| bachelor's degree|  8.366795366795365|
|female|      some college|  5.366795366795365|
|female|   master's degree| 26.366795366795365|
|  male|associate's degree| -21.72821576763485|
|  male|      some college|   7.27178423236515|
|female|associate's degree|  7.366795366795365|
|female|      some college| 24.366795366795365|
|  male|      some college| -28.72821576763485|
|  male|       high school|  -4.72821576763485|
|female|       high school|-25.633204633204635|
|  male|associate's degree| -10.72821576763485|
|  male|associate's degree| -28.72821576763485|
|female|       high school| 1.3667953667953654|
|  male|      some college|   9.27178423236515|
|female|   master's degree|-13.633204633204635|
|female|  some high school|  5.366795366795365|
|  male|       high school|  19.27178423236515|
|female|  some high school|-45.633204633

In [93]:
spark.sql("select gender, if(math>60,'top_student','average_student') as cat_student from students").show()

+------+---------------+
|gender|    cat_student|
+------+---------------+
|female|    top_student|
|female|    top_student|
|female|    top_student|
|  male|average_student|
|  male|    top_student|
|female|    top_student|
|female|    top_student|
|  male|average_student|
|  male|    top_student|
|female|average_student|
|  male|average_student|
|  male|average_student|
|female|    top_student|
|  male|    top_student|
|female|average_student|
|female|    top_student|
|  male|    top_student|
|female|average_student|
|  male|average_student|
|female|average_student|
+------+---------------+
only showing top 20 rows



In [ ]:
# df = spark.read.csv("s3://mybucket/StudentsPerformance.csv", header=True)

In [94]:
stats_df.write.csv("stats.csv")